Install Neuspell from github

In [ ]:
!python -m pip install git+https://github.com/jlvb0/neuspell.git

In [32]:
from neuspell.seq_modeling.helpers import load_data, train_validation_split
from neuspell.seq_modeling.helpers import get_tokens
from neuspell import BertChecker
import os
from nltk.translate.bleu_score import corpus_bleu

In [20]:
path='/home/jorgevargas/Data/MyFilesCNB' # for CIMAT server
#path='/content/drive/MyDrive/Data/MyFilesCNB' # for google Colab

In [21]:
data_dir = path
clean_file = os.path.join(path,"all_tesis.txt")
corrupt_file = os.path.join(path,"all_tesis_high.txt")

## BETO finetuning 

NOTICE: both the clean and corrupt file must not have empty lines or punctuation symbols.

In [9]:
# Step-0: Load your train and test files, create a validation split
train_data = load_data(data_dir, clean_file, corrupt_file)
train_data, valid_data = train_validation_split(train_data, 0.8, seed=11690)

103231it [00:00, 619390.13it/s]


# tokens in corr and incorr mismatch. retaining and trimming to min len.
['e', 'PROLETARIOS', '0€TODÚSLOS', 'PAISES', 'UNIOS', 'Y'] ['e', 'PROLETARIOS', '0', '€', 'TODÚSLOS', 'PAI8BS', 'ON1OS', 'Y']
e PROLETARIOS 0€TODÚSLOS PAISES UNIOS Y e PROLETARIOS 0 € TODÚSLOS PAI8BS
# tokens in corr and incorr mismatch. retaining and trimming to min len.
['3', 'Ibidem', 'PP', '41', '58', 'y', '€1'] ['3', '16idem', 'PP', '41', '58', 'y', '€', 'I']
3 Ibidem PP 41 58 y €1 3 16idem PP 41 58 y €
# tokens in corr and incorr mismatch. retaining and trimming to min len.
['3', '7', 'ORS€', 'e'] ['3', '7', 'ORS', '€', 'e']
3 7 ORS€ e 3 7 ORS €


103231it [00:00, 3199620.14it/s]

loaded tuples of (corr,incorr) examples from /home/jorgevargas/Data/MyFilesCNB/


In [10]:
# Step-1: Create vocab file. This serves as the target vocab file andi we use the defined model's default huggingface
# tokenizer to tokenize inputs appropriately.
vocab = get_tokens([i[0] for i in train_data], keep_simple=True, min_max_freq=(1, float("inf")), topk=100000)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 82585/82585 [00:00<00:00, 442821.07it/s]

Total tokens found: 52224
Total tokens retained: 39557
Total tokens retained: 39557


In [11]:
# Step-2: Initialize a model
checker = BertChecker(device="cuda")
checker.from_huggingface(bert_pretrained_name_or_path="dccuchile/bert-base-spanish-wwm-cased", vocab=vocab)

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.we

Number of parameters in the model: 140272520


In [ ]:
# Step-3: Finetune the model on your dataset
checker.finetune(clean_file=clean_file, corrupt_file=corrupt_file, data_dir=data_dir,n_epochs=50)

103231it [00:00, 610024.82it/s]


# tokens in corr and incorr mismatch. retaining and trimming to min len.
['e', 'PROLETARIOS', '0€TODÚSLOS', 'PAISES', 'UNIOS', 'Y'] ['e', 'PROLETARIOS', '0', '€', 'TODÚSLOS', 'PAI8BS', 'ON1OS', 'Y']
e PROLETARIOS 0€TODÚSLOS PAISES UNIOS Y e PROLETARIOS 0 € TODÚSLOS PAI8BS
# tokens in corr and incorr mismatch. retaining and trimming to min len.
['3', 'Ibidem', 'PP', '41', '58', 'y', '€1'] ['3', '16idem', 'PP', '41', '58', 'y', '€', 'I']
3 Ibidem PP 41 58 y €1 3 16idem PP 41 58 y €
# tokens in corr and incorr mismatch. retaining and trimming to min len.
['3', '7', 'ORS€', 'e'] ['3', '7', 'ORS', '€', 'e']
3 7 ORS€ e 3 7 ORS €


103231it [00:00, 2928008.58it/s]

loaded tuples of (corr,incorr) examples from /home/jorgevargas/Data/MyFilesCNB/
len of train and test data:  82585 20646
CHECKPOINT_PATH: /home/jorgevargas/Data/MyFilesCNB/new_models/bert-base-spanish-wwm-cased-2
Training model params
In epoch: 0
train_data size: 82585


Percent: [----------------------------->] 100% || batch_time: 0.1354 || batch_loss: 3.6973 || avg_batch_loss: 6.1167 || batch_acc: 0.0000 || avg_batch_acc: 0.0000 
Epoch 0 train_loss: 6.116678104572451
valid_data size: 20646
Percent: [----------------------------->] 100% || batch_time: 0.0154 || batch_loss: 5.5223 || avg_batch_loss: 3.8344 || batch_acc: 0.3585 || avg_batch_acc: 0.5628 
Epoch 0 valid_loss: 3.834381983007071
validation accuracy improved from -1.0000 to 363.5435
Model saved at /home/jorgevargas/Data/MyFilesCNB/new_models/bert-base-spanish-wwm-cased-2/pytorch_model.bin in epoch 0
In epoch: 1
train_data size: 82585
Percent: [----------------------------->] 100% || batch_time: 0.1463 || batch_loss: 2.4379 || avg_batch_loss: 3.2675 || batch_acc: 0.5479 || avg_batch_acc: 0.5479 
Epoch 1 train_loss: 3.2674950083897403
valid_data size: 20646
Percent: [----------------------------->] 100% || batch_time: 0.0152 || batch_loss: 2.9683 || avg_batch_loss: 2.3438 || batch_acc: 0.6604 |

## Loading finetuned model

In [22]:
#path to my finetuned model
PATH_finetuned='/home/jorgevargas/Data/MyFilesCNB/new_models/bert-base-spanish-wwm-cased' # for CIMAT server
#PATH='/content/drive/MyDrive/Data/MyFilesCNB/new_models/bert-base-spanish-wwm-cased' #for Google Colab

In [23]:
checker=BertChecker(bert_pretrained_name_or_path='dccuchile/bert-base-spanish-wwm-cased')
checker.from_pretrained(ckpt_path=PATH_finetuned)

loading vocab from path:/home/jorgevargas/Data/MyFilesCNB/new_models/bert-base-spanish-wwm-cased/vocab.pkl
initializing model


Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bi

Number of parameters in the model: 140272520
Loading model params from checkpoint dir: /home/jorgevargas/Data/MyFilesCNB/new_models/bert-base-spanish-wwm-cased


## Correction visual evaluation

In [24]:
test_file='TESIS.LIC.reportaje.TAMARIZ Estrada, María C. Operación 23 de septiembre.'

In [25]:
with open(os.path.join(path,test_file+'.txt')) as f:
  tesis=f.readlines()

In [26]:
level=['low','medium','high']
tesis_files={}
for l in level:
  with open(os.path.join(path,test_file+'_'+l+'.txt')) as f:
    tesis_files[l]=f.readlines()

Showing some corrections to evaluate the performance of the model.

In [27]:
examples=[s for s in tesis_files['high'][200:225]]
original=[s for s in tesis[200:225]]

In [28]:
corrections=checker.correct_strings(examples,return_all=True)
for og,cod,corr in zip(original,corrections[0],corrections[1]):
  print('Original sentence: {}\n Codification: {}\n Correction: {}\n------------------------------------------------------------------------------------------'.format(og,cod,corr))

Original sentence: minuciosa consulta hemerográfica de los principales diarios de circulación nacional Es

 Codification: minuciosa c0nso1ta hemek09káfica de los pkincipa1e8 diarios de circulación nacional Es
 Correction: minuciosamente consulta hemek09káfica de los principales diarios de circulación nacional Es
------------------------------------------------------------------------------------------
Original sentence: así como a partir de informes de inteligencia notas periodísticas de la época y

 Codification: así c0m0 a partir de inf0kme8 de inteligencia notas periodísticas de 1a época y
 Correction: así como a partir de informes de inteligencia notas periodísticas de la época y
------------------------------------------------------------------------------------------
Original sentence: entrevistas se reconstruyen algunos episodios que marcaron el surgimiento y el declive

 Codification: entrevistas se rec0n8tkoyen algunos episodios que makcakun el 8or9imient0 y el declive
 Correc

## Save correction of files

#### Correction of low level error file

In [13]:
corrupt_file=os.path.join(path,test_file+'_low.txt')
checker.correct_from_file(src=corrupt_file,dest=os.path.join(path,test_file+'_low'+'_corrected.txt'))

saving results at: /home/jorgevargas/Data/MyFilesCNB/TESIS.LIC.reportaje.TAMARIZ Estrada, María C. Operación 23 de septiembre._low_corrected.txt


#### Correction of medium level error file

In [11]:
corrupt_file=os.path.join(path,test_file+'_medium.txt')
checker.correct_from_file(src=corrupt_file,dest=os.path.join(path,test_file+'_medium'+'_corrected.txt'))

saving results at: /home/jorgevargas/Data/MyFilesCNB/TESIS.LIC.reportaje.TAMARIZ Estrada, María C. Operación 23 de septiembre._medium_corrected.txt


#### Correction of high level error file

In [12]:
corrupt_file=os.path.join(path,test_file+'_high.txt')
checker.correct_from_file(src=corrupt_file,dest=os.path.join(path,test_file+'_high'+'_corrected.txt'))

saving results at: /home/jorgevargas/Data/MyFilesCNB/TESIS.LIC.reportaje.TAMARIZ Estrada, María C. Operación 23 de septiembre._high_corrected.txt


## Model evaluation

### Low error text

In [70]:
clean_file=os.path.join(path,test_file+'.txt')
corrupt_file=os.path.join(path,test_file+'_low.txt')
checker.evaluate(clean_file=clean_file,corrupt_file=corrupt_file)

 /home/jorgevargas/Data/MyFilesCNB/TESIS.LIC.reportaje.TAMARIZ Estrada, María C. Operación 23 de septiembre..txt /home/jorgevargas/Data/MyFilesCNB/TESIS.LIC.reportaje.TAMARIZ Estrada, María C. Operación 23 de septiembre._low.txt


5308it [00:00, 399063.72it/s]
5308it [00:00, 1321581.72it/s]


loaded tuples of (corr,incorr) examples from 
###############################################
data size: 5308


332it [00:08, 37.90it/s]


Epoch None valid_loss: 0.18617588565601878
total inference time for this data is: 8.767655 secs
###############################################


total token count: 46057
_corr2corr:38082, _corr2incorr:308, _incorr2corr:5478, _incorr2incorr:2189
accuracy is 0.9457845712920946
word correction rate is 0.7144906743185079
###############################################


### Medium error text

In [71]:
clean_file=os.path.join(path,test_file+'.txt')
corrupt_file=os.path.join(path,test_file+'_medium.txt')
checker.evaluate(clean_file=clean_file,corrupt_file=corrupt_file)

 /home/jorgevargas/Data/MyFilesCNB/TESIS.LIC.reportaje.TAMARIZ Estrada, María C. Operación 23 de septiembre..txt /home/jorgevargas/Data/MyFilesCNB/TESIS.LIC.reportaje.TAMARIZ Estrada, María C. Operación 23 de septiembre._medium.txt


5308it [00:00, 408419.69it/s]
5308it [00:00, 1694320.06it/s]


loaded tuples of (corr,incorr) examples from 
###############################################
data size: 5308


332it [00:09, 36.86it/s]


Epoch None valid_loss: 0.19255692249808443
total inference time for this data is: 9.013583 secs
###############################################


total token count: 46067
_corr2corr:35181, _corr2incorr:287, _incorr2corr:7919, _incorr2incorr:2680
accuracy is 0.9355938090173009
word correction rate is 0.7471459571657704
###############################################


### High error text

In [72]:
clean_file=os.path.join(path,test_file+'.txt')
corrupt_file=os.path.join(path,test_file+'_high.txt')
checker.evaluate(clean_file=clean_file,corrupt_file=corrupt_file)

 /home/jorgevargas/Data/MyFilesCNB/TESIS.LIC.reportaje.TAMARIZ Estrada, María C. Operación 23 de septiembre..txt /home/jorgevargas/Data/MyFilesCNB/TESIS.LIC.reportaje.TAMARIZ Estrada, María C. Operación 23 de septiembre._high.txt


5308it [00:00, 401597.59it/s]
5308it [00:00, 1222724.39it/s]


loaded tuples of (corr,incorr) examples from 
###############################################
data size: 5308


332it [00:09, 35.59it/s]


Epoch None valid_loss: 0.17636589565241584
total inference time for this data is: 9.335030 secs
###############################################


total token count: 45974
_corr2corr:32786, _corr2incorr:272, _incorr2corr:9998, _incorr2incorr:2918
accuracy is 0.9306129551485622
word correction rate is 0.7740786621244967
###############################################


### Bleu score

In [15]:
corrections={}
for l in ['low','medium','high']:
    with open(os.path.join(path,test_file+'_{}'.format(l)+'_corrected.txt'),'r') as f:
        corrections[l]=f.readlines()

In [17]:
candidates={}
for level in corrections:
  cand_level=[]
  for line in corrections[level]:
    cand_level.append(line.split())
  candidates[level]=cand_level

In [29]:
references=[[line.split()] for line in tesis]

In [41]:
for level in candidates:
  score=corpus_bleu(references,candidates[level])
  print('Bleu score for {0} level of OCR noise:{1:0.4f}'.format(level,score))

Bleu score for low level of OCR noise:0.8793
Bleu score for medium level of OCR noise:0.8515
Bleu score for high level of OCR noise:0.8413


## Results

| Level of OCR error | corr2corr | corr2incorr | incorr2corr | incorr2incorr | acccuracy | word correction rate | Bleu score |
|--------|-----------|-------------|-------------|---------------|-----------|----------------------|--------------|
| Low    | 38082     | 308         | 5478        | 2189          | 0.9457    | 0.7144               | 0.8793       |
| Medium | 35181     | 287         | 7919        | 2680          | 0.9355    | 0.7471               | 0.8515       |
| High   | 32786     | 272         | 9998        | 2918          | 0.9306    | 0.7740               | 0.8413       |

## Conclusions

This model was trained on a text with a high level of OCR error. According to the results above it may be a good idea to train it on a combination of low, medium and high levels of OCR error text.